In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

In [ ]:
from src.constants import *
from src.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([Path(self.config.artifacts_root)])

    

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([Path(config.root_dir)])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [ ]:
import os
from src.logging import logger
from transformers import AutoTokenizer
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    ## validate all files are present, and add the value to the status file

    def convert_examples_to_features(self, examples):
        input_encodings = self.tokenizer(
            examples['dialogue'], max_length=512, padding='max_length', truncation=True
        )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(
                examples['summary'], max_length=128, padding='max_length', truncation=True
            )
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids'],
        }
    
    def convert(self):
        dataset_samsum =load_from_disk(self.config.data_path)
        logger.info(f"Dataset loaded from {self.config.data_path}")
        tokenized_datasets = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        tokenized_datasets.save_to_disk(self.config.root_dir)
        logger.info(f"Tokenized dataset saved to {self.config.root_dir}")

In [ ]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e